In [1]:
from bs4 import BeautifulSoup
import numpy as np
import requests

In [2]:
request = requests.get("https://www.biznesradar.pl/wskazniki-wartosci-rynkowej/GIG", allow_redirects=False)
html_code = BeautifulSoup(request.content, 'html.parser')


In [42]:
class CompanyIndexScraper:
    SKELETON_URL = "https://www.biznesradar.pl"
    INDEXES_TO_SCRAP = {
        "wskazniki-wartosci-rynkowej" : [
            "Cena / Wartość księgowa",
            "Cena / Wartość księgowa Grahama",
            "Cena / Przychody ze sprzedaży",
            "Cena / Zysk",
            "Cena / Zysk operacyjny",
            "EV / Przychody ze sprzedaży",
            "EV / EBIT",
            "EV / EBITDA",
        ],
        "wskazniki-rentownosci" : [
            "ROE",
            "ROA",
            "Marża zysku operacyjnego",
            "Marża zysku netto",
            "Marża zysku ze sprzedaży",
            "Marża zysku brutto",
            "Marża zysku brutto ze sprzedaży",
            "Rentowność operacyjna aktywów",
        ],
        "wskazniki-przeplywow-pienieznych" : [
            "Udział zysku netto w przepływach operacyjnych",
            "Wskaźnik źródeł finansowania inwestycji",
        ],
        "wskazniki-zadluzenia" : [
            "Zadłużenie ogólne",
            "Zadłużenie kapitału własnego",
            "Zastosowanie kapitału obcego",
            "Wskaźnik ogólnej sytuacji finansowej",
        ],
        "wskazniki-plynnosci" : [
            "I stopień pokrycia",
            "Płynność gotówkowa",
            "Płynność szybka",
            "Płynność bieżąca",
            "Pokrycie zobowiązań należnościami",
        ],
    }

    DATA_TYPE = [
        "Company",
        "Sector",
        "CompanyY/Y",
        "SectorY/Y",
        "CompanyQ/Q",
        "SectorQ/Q",
    ]

    def __init__(self):
        self.dates_header_ = None
        self.html_table_   = None
        self.data_array_   = None
        self.columns_name_ = sum(CompanyIndexScraper.INDEXES_TO_SCRAP.values(), [])

    def __generate_url_address(self, indexes_type : str, ticker : str) -> str:
        return f"{CompanyIndexScraper.SKELETON_URL}/{indexes_type}/{ticker}"
    
    def __download_http_response(self, url : str) -> BeautifulSoup:
        request = requests.get(url, allow_redirects=False)
        # If ticker is different or if it is shortcuts
        if request.status_code == 301:
            url = f"{CompanyIndexScraper.SKELETON_URL}{request.headers['Location']}"
            request = requests.get(url)
        # If some error with request raise exception
        if request.status_code != 200:
            raise Exception(f"Download http response error code: {request.status_code} on address {url}")
        return BeautifulSoup(request.content, 'html.parser')
         
    def __download_and_prepare_html_table(self, indexes_key : str, ticker : str) -> None:
        url = self.__generate_url_address(indexes_key, ticker)
        html_response = self.__download_http_response(url)
        self.html_table_ = html_response.find("table", attrs={"class":"report-table"}) 

    def __scrap_header_date(self):
        header_row = self.html_table_.find("tr")
        self.dates_header_ = {"Date": ["".join(date.text.split()).split('(')[0] for date in header_row.find_all("th", attrs={"class":"thq h"})]}

    def __prepare_data_space(self) -> None:
        # Before use dates_header_ should be initialized 
        self.data_array_ = np.empty((len(self.dates_header_["Date"]), len(self.columns_name_), len(CompanyIndexScraper.DATA_TYPE)))
        self.data_array_[:] = np.nan
    
    def __scrap_data_table(self, index_key : str) -> None:
        data_rows = self.html_table_.find_all("tr")[1:] # Omit header line
        for row in data_rows:
            self.__scrap_data_row(index_key, row) 

    def __scrap_data_row(self, index_key : str, row : str) -> None:
        # Check is row to scrap 
        row.find("ts", attrs={"class":"f"}).text
        

    def scrap_data(self, ticker : str):
        for index_key in CompanyIndexScraper.INDEXES_TO_SCRAP.keys():
            self.__download_and_prepare_html_table(index_key, ticker)
            self.__scrap_header_date()
            self.__prepare_data_space()
            break



In [43]:
cis = CompanyIndexScraper()
cis.scrap_data("ATR")